TODO: test batch size other than 1

# Preparation

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from IPython.display import clear_output
from sklearn.metrics import r2_score
from BinvoxDataset import CustomDataset
from Networks import ConvNetScalarLabel, count_parameters
import time

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: changli_824 (additive-parts). Use `wandb login --relogin` to force relogin


True

# Calculate number of parameters

In [4]:
count_parameters(ConvNetScalarLabel())

+--------------------+------------+
|      Modules       | Parameters |
+--------------------+------------+
| layers.0.0.weight  |     54     |
|  layers.0.0.bias   |     2      |
| layers.0.1.weight  |     2      |
|  layers.0.1.bias   |     2      |
| layers.2.0.weight  |    216     |
|  layers.2.0.bias   |     4      |
| layers.2.1.weight  |     4      |
|  layers.2.1.bias   |     4      |
| layers.4.0.weight  |    864     |
|  layers.4.0.bias   |     8      |
| layers.4.1.weight  |     8      |
|  layers.4.1.bias   |     8      |
| layers.6.0.weight  |    6912    |
|  layers.6.0.bias   |     32     |
| layers.6.1.weight  |     32     |
|  layers.6.1.bias   |     32     |
| layers.8.0.weight  |   110592   |
|  layers.8.0.bias   |    128     |
| layers.8.1.weight  |    128     |
|  layers.8.1.bias   |    128     |
| layers.10.0.weight |   884736   |
|  layers.10.0.bias  |    256     |
| layers.10.1.weight |    256     |
|  layers.10.1.bias  |    256     |
|  linear_1.weight   |    40

1008793

# Create dataset

In [5]:
def transform(voxel):
    # return torch.unsqueeze(torch.tensor(condense_voxel_array(voxel, 64), dtype = torch.float32), 0)
    return torch.unsqueeze(torch.tensor(voxel, dtype = torch.float32), 0)

In [6]:
import json
configs = json.load(open('config.json', 'r'))
train_folder_path = configs['train_folder_path']
test_folder_path = configs['test_folder_path']
train_folder_name = configs['train_folder_name']
test_folder_name = configs['test_folder_name']
label_file_path = configs['label_file_path']
label_type = configs['label_type']

In [7]:
# Small values of max_count is for debugging and testing. Use max_count = None for full runs for training data.
dataset = CustomDataset(input_folder_path = train_folder_path, input_folder_name = train_folder_name, label_file_path = label_file_path, transform = transform, max_count = None, ram_limit = 1000, label_type = label_type)
dataset_limited = CustomDataset(input_folder_path = test_folder_path, input_folder_name = test_folder_name, label_file_path = label_file_path, transform = transform, max_count = None, ram_limit = 1000, label_type = label_type)

In [8]:
len(dataset)

22258

In [9]:
len(dataset_limited)

13943

# Define Training Logic

In [10]:
def train_epoch(model, training_loader, optimizer, loss_fn):
    cumulative_loss = 0.0
    for i, data in enumerate(training_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = torch.squeeze(labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Make predictions
        outputs = model(inputs)

        # Compute loss and its gradients
        # print('label shape', labels.shape)
        # print('outputs:', outputs)
        # print('labels:', labels)
        loss = loss_fn(outputs, labels.float())
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        cumulative_loss += loss.item()
        
        wandb.log({'batch loss': loss.item()})
    return cumulative_loss / len(training_loader), cumulative_loss

In [11]:
def train(config, loss_fn):
    clear_output(wait = True)
    
    # initialize a wandb run
    wandb.init(config = config)

    # copy the config
    config = wandb.config
    
    print('config:', config)

    # get training loader
    training_loader = DataLoader(dataset, batch_size = config.batch_size, shuffle = False)

    # initialize model
    if config.activation_fn == 'ReLU':
        activation_fn = nn.ReLU()
    
    if config.activation_fn == 'Sigmoid':
        activation_fn = nn.Sigmoid()
    
    model = ConvNetScalarLabel(kernel_size = config.kernel_size, activation_fn = activation_fn).to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = config.learning_rate, momentum = 0.9)

    for epoch in range(config.epochs_choice):
        tic = time.time()
        avg_loss_per_batch, cumulative_loss = train_epoch(model, training_loader, optimizer, loss_fn)
        toc = time.time()
        wandb.log({'avg_loss_per_batch': avg_loss_per_batch, 'cumulative_loss': cumulative_loss, 'time': toc - tic})
        print(f'Loss for epoch {epoch}: {cumulative_loss}, time for epoch {epoch}: {toc - tic}')
    
    return model

In [12]:
def test(config, model, loss_fn):
    # copy the config
    config = wandb.config
    
    # get testing loader
    testing_loader = DataLoader(dataset_limited, batch_size = config.batch_size, shuffle = False)
    
    testing_loss = 0.0
    y_true = []
    y_pred = []
    for i, data in enumerate(testing_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.float())
        testing_loss += loss.item()

        y_true.extend(labels.cpu().numpy().tolist())
        y_pred.extend(outputs.cpu().detach().numpy().tolist())
    return testing_loss / len(testing_loader), testing_loss, r2_score(y_true = y_true, y_pred = y_pred)

In [13]:
def evaluate(config = None):
    loss_fn = nn.MSELoss()
    model = train(config, loss_fn)
    torch.save(model, 'model.pt')
    avg_loss_per_batch_test, testing_loss, r2 = test(config, model, loss_fn)
    wandb.log({'avg_loss_per_batch_test': avg_loss_per_batch_test, 'testing_loss': testing_loss, 'r2': r2})

# Training settings

In [14]:
sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'testing_loss',
    'goal': 'minimize'
    }
sweep_config['metric'] = metric
parameters_dict = {
    'kernel_size': {
        'values': [3, 4, 5]
    },
    'activation_fn': {
        'values': ['ReLU', 'Sigmoid']
    },
    'epochs_choice': {
          'values': [5, 10, 20]
    },
    'learning_rate': {
        'values': [1e-4, 1e-3, 1e-2]
    },
    'batch_size': {
        'values': [4]
    },
}

parameters_dict = {
    'kernel_size': {
        'values': [3]
    },
    'activation_fn': {
        'values': ['ReLU']
    },
    'epochs_choice': {
          'values': [5]
    },
    'learning_rate': {
        'values': [1e-4]
    },
    'batch_size': {
        'values': [4]
    },
}

sweep_config['parameters'] = parameters_dict

# Start

In [15]:
sweep_id = wandb.sweep(sweep_config, project = 'CNN_sweep_scalar')

Create sweep with ID: hfq037fx
Sweep URL: https://wandb.ai/additive-parts/CNN_sweep_scalar/sweeps/hfq037fx


In [ ]:
wandb.agent(sweep_id = sweep_id, function = evaluate)

config: {'activation_fn': 'ReLU', 'batch_size': 4, 'epochs_choice': 5, 'kernel_size': 3, 'learning_rate': 0.0001}
Loading samples 0 through 999
Processing sample number 0
Loading samples 1000 through 1999
Processing sample number 1000
Loading samples 2000 through 2999
Processing sample number 2000
Loading samples 3000 through 3999
Processing sample number 3000
Loading samples 4000 through 4999
Processing sample number 4000
Loading samples 5000 through 5999


Exception in thread Thread-25:
Traceback (most recent call last):
  File "C:\Users\Public\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Public\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Public\anaconda3\lib\site-packages\multiprocess\pool.py", line 513, in _handle_workers
    cls._maintain_pool(ctx, Process, processes, pool, inqueue,
  File "C:\Users\Public\anaconda3\lib\site-packages\multiprocess\pool.py", line 337, in _maintain_pool
    Pool._repopulate_pool_static(ctx, Process, processes, pool,
  File "C:\Users\Public\anaconda3\lib\site-packages\multiprocess\pool.py", line 326, in _repopulate_pool_static
    w.start()
  File "C:\Users\Public\anaconda3\lib\site-packages\multiprocess\process.py", line 121, in start
    self._popen = self._Popen(self)
  File "C:\Users\Public\anaconda3\lib\site-packages\multiprocess\context.py", line 327, in _Popen
    return Popen(process_obj)